In [8]:
# simple compiler for 8-bit computer

INSTRUCTIONS = {
    "nop": 0b00000000,
    "lda": 0b00010000,
    "add": 0b00100000,
    "sub": 0b00110000,
    "sta": 0b01000000,
    "ldi": 0b01010000,
    "jmp": 0b01100000,
    "jc": 0b01110000,
    "jz": 0b10000000,
    "out": 0b11100000,
    "hlt": 0b11110000,
    "var": None,  # special directive for defining data
}


def assemble(assembly_code):
    lines = assembly_code.strip().splitlines()
    machine_code = [0] * 16  # Initialize memory with 256 bytes
    pc = 0  # Program counter
    vars = {}
    var_idx = 15

    for line in lines:
        parts = line.split()
        if len(parts) == 0:
            continue
        instr = parts[0]#.upper()
        if instr not in INSTRUCTIONS and ":" not in instr:
            raise ValueError(f"Unknown instruction: {instr}")

        if instr == "var":
            # Handle variable definition: VAR ADDR VALUE
            if len(parts) != 3:
                raise ValueError("'var' requires address and value")
            variable_name =  parts[1]
            value = int(parts[2])
            
            if value < 0 or value > 255:
                raise ValueError(f"Value out of range (0-255): {value}")
            
            vars[variable_name] = {"value": value, "idx": var_idx}
            machine_code[var_idx] = value
            var_idx -= 1
        elif ":" in instr:
            jump_adress = instr.split(":")[0]
            print(jump_adress)
            
        
        else:
            # Handle regular instructions - opcode and operand in separate bytes
            opcode = INSTRUCTIONS[instr]
            machine_code[pc] = opcode
            pc += 1

            # If instruction has an operand, store it as the next byte
            if len(parts) > 1:
                variable_name = parts[1]
                operand = vars[variable_name]["idx"]
                if operand < 0 or operand > 255:
                    raise ValueError(f"Operand out of range (0-255): {operand}")
                machine_code[pc] = operand
                pc += 1
        print(machine_code)

    return machine_code


if __name__ == "__main__":
    sample_code = """
        var iter 1
        var product 0
        var x 8
        var y 8
    main:
        lda x
        sub iter
        jc loop
    end:
        lda product
        out
        hlt
    loop:
        sta x
        lda product
        add y
        sta product
        jmp main
    """
    machine_code = assemble(sample_code)
    i = 0
    while i < len(machine_code):
        code = machine_code[i]
        if code == 0:
            print(f"0b{code:08b}, // {i} - NOP")
            i += 1
        else:
            # Find instruction name
            instr_name = next((name for name, op in INSTRUCTIONS.items() if op == code), None)
            if instr_name:
                # Check if next byte is an operand
                if i + 1 < len(machine_code) and instr_name in ["lda", "add", "sub", "sta", "ldi", "jmp", "jc", "jz"]:
                    operand = machine_code[i + 1]
                    print(f"0b{code:08b}, // {i} - ISTR: {instr_name}")
                    print(f"0b{operand:08b}, // {i + 1} - MEM: {operand}")
                    i += 2
                else:
                    print(f"0b{code:08b}, // {i} - {instr_name}")
                    i += 1
            else:
                print(f"0b{code:08b}, // {i} - DATA")
                i += 1


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 0, 1]
main
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 0, 1]
[16, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 0, 1]
[16, 13, 48, 15, 0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 0, 1]


KeyError: 'loop'